In [ ]:
import os
# Local Imports
import coastseg
from coastseg import coastseg_map
from coastseg import coastseg_logs #must be the first module loaded to create logs folder

# External Imports
import ee
from google.auth import exceptions as google_auth_exceptions

# suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Authenticate and Initialize with Google Earth Engine (GEE)

- Run this cell to initialize with GEE which will allow you to download remote sensing data from GEE.

### First Time Users

- In order to use Google Earth Engine (GEE) you will need to sign up to request access to use Google Earth Engine.https://signup.earthengine.google.com. You will only need to do this once and it takes only a day to get your account verified.

### How `ee.Authenticate()` works

- In order to initialize with GEE you will need an authorization token with is obtained by running `ee.Authenticate()`.This token lasts 7 days and during those 7 days you will not need to authenticate with google earth engine with an access code. Once the 7 days are up you will need to reauthenticate to use GEE again.


In [ ]:
try:
    ee.Initialize()
except google_auth_exceptions.RefreshError as exception:
    print("Please authenticate with Google:\n")
    ee.Authenticate()
    ee.Initialize()

# How to Use The Map

---
1.  Click `Save Settings` Button
1. Use the rectangle tool to draw a bounding box along the coastline.
2. Click `Generate ROI` to create ROI rectangles along the coastline in the bounding box. This may take some time.
- You should see a coastline appear in yellow and some rectangles along it.
3. Load transects into your bounding box with the `Load Transects` button. If any exist for the bounding box you selected they should appear.
4. Click the ROIs you want to download.
5. Once you've selected all the ROIs you want to download click `Downlod Imagery`
 - If any of the ROIs succesfully download they will have their own folder with all their data in the `data` directory in the `CoastSeg` directory
6. To extract a timeseries of shorelines for the ROIs you downloaded click `Extract Shorelines`
7. If any shorelines were extracted you can compute the cross distances along the transects and shoreline by clicking `Compute Transects` and save the output to a json file in the roi directory
8. If any shorelines were extracted you can compute the cross distances along the transects and shoreline by clicking `Save Transects CSV`  and save the output to a csv in the roi directory

In [ ]:
from src.coastseg.map_UI import UI
coastsegmap=coastseg_map.CoastSeg_Map()
coastseg_ui = UI(coastsegmap)
coastseg_ui.create_dashboard()

In [ ]:
import os 

In [ ]:
outfile=r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\zox.geojson'

In [ ]:
os.path.isfile(outfile)

## Run Models on Imagery
1. Select if you want to use a GPU or not. By default you won't your computer's GPU
2. Select whether your data is the type RGB or MNDWI
3. Select the model you want to use
4. Select Ensemble or Best (Ensemble is recommended) 


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from src.coastseg.UI_models import UI_Models
models_ui = UI_Models()
models_ui.create_dashboard()

In [ ]:
import os
import glob
import asyncio
import platform
import json
import tqdm
# from tqdm import tqdm
import tqdm.asyncio
import aiohttp
from coastseg import zoo_model

In [ ]:
import asyncio
import random


async def produce(queue, n):
    for x in range(1, n + 1):
        # produce an item
        print('producing {}/{}'.format(x, n))
        # simulate i/o operation using sleep
        await asyncio.sleep(random.random())
        item = str(x)
        # put the item in the queue
        await queue.put(item)

    # indicate the producer is done
    await queue.put(None)


async def consume(queue):
    while True:
        # wait for an item from the producer
        item = await queue.get()
        if item is None:
            # the producer emits None to indicate that it is done
            break

        # process the item
        print('consuming {}'.format(item))
        # simulate i/o operation using sleep
        await asyncio.sleep(random.random())


loop = asyncio.get_event_loop()
queue = asyncio.Queue()
asyncio.ensure_future(produce(queue, 10), loop=loop)
loop.run_until_complete(consume(queue))

In [ ]:
# if platform.system()=='Windows':
#         asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        
async def fetch(session,url:str,save_path: str):
    chunk_size: int = 128
    async with session.get(url,raise_for_status=True) as r:
        with open(save_path, "wb") as fd:
            async for chunk in r.content.iter_chunked(chunk_size):
                fd.write(chunk)

async def fetch_all(session,url_dict):
    print(url_dict.keys())
    tasks = []
    for save_path,url in url_dict.items():
        task = asyncio.create_task(fetch(session,url,save_path)) 
        tasks.append(task)
    await tqdm.asyncio.tqdm.gather(*tasks)
    # await asyncio.gather(*tasks)

async def async_download_urls(url_dict:dict)->None:
    print(url_dict.keys())
    async with aiohttp.ClientSession() as session:
        await fetch_all(session,url_dict)

In [ ]:

url_dict = {'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v1_fullmodel.h5': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_513_v1_fullmodel.h5', 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v1.json': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v1.json', 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v2_fullmodel.h5': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v2_fullmodel.h5', 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v2.json': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v2.json', 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v3_fullmodel.h5': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v3_fullmodel.h5', 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v3.json': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v3.json', 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v4_fullmodel.h5': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v4_fullmodel.h5', 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v4.json': 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v4.json'}

In [ ]:
def run_async_download(url_dict:dict):
    if platform.system()=='Windows':
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    loop = asyncio.get_event_loop()
    loop.create_task(zoo_model.async_download_urls(url_dict))

In [ ]:
run_async_download(url_dict)

In [ ]:
async def async_download_urls(url_dict:dict)->None:
    print(url_dict.keys())
    await fn(1)
    # async with aiohttp.ClientSession() as session:
    #     await  fn(1)

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(async_download_urls(url_dict))

In [ ]:
import asyncio
async def fn(num):
  print('hello',1)
  await asyncio.sleep(1)
  print('world')

await fn(1)

In [ ]:
from tqdm.asyncio import trange, tqdm
async for i in trange(10):
    await asyncio.sleep(1)

In [ ]:
loop = asyncio.get_event_loop()
loop

In [ ]:
loop.create_task(fn(1))

In [ ]:
filename= 'C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v1_fullmodel.h5'.split(os.sep)[-1]
filename

In [ ]:
json_filename='C:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\src\\coastseg\\downloaded_models\\s2-landsat78-4class_6950472\\sat4class_rgb_512_v1_fullmodel.h5'.replace("fullmodel.h5",".json")
json_filename

In [ ]:
url = 'https://www.petco.com/'
with requests.get(url, stream=True) as r:
    print(r)
    print(r.headers)

In [ ]:
url = 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_mndwi_512_v7.json'
save_path  = r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\src\coastseg\downloaded_models\s2-landsat78-4class_6950472\sat4class_mndwi_512_v7.json'
chunk_size = 128
with requests.get(url, stream=True) as r:
    print(r)
    print(r.headers.keys)
    print(r.headers.get("Content-Length"))
    print(type(r.headers.get("Content-Length")))
    print(type(total_content_length) == str)
    total_content_length = r.headers.get("Content-Length")
    if total_content_length is None:
            with open(save_path, "wb") as fd:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    fd.write(chunk)
    elif type(total_content_length) == str:
        total_content_length = int(total_content_length)
        with open(save_path, "wb") as fd:
            with tqdm(
                    total=total_content_length ,
                    unit="B",
                    unit_scale=True,
                    unit_divisor=1024,
                    desc="Downloading Model",
                    initial=0,
                    ascii=True,
                ) as pbar:
                    for chunk in r.iter_content(chunk_size=chunk_size):
                        fd.write(chunk)
                        pbar.update(len(chunk))

In [ ]:
url = 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v1_fullmodel.json'
save_path  = r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\src\coastseg\downloaded_models\s2-landsat78-4class_6950472\sat4class_rgb_512_v1_fullmodel.json'
chunk_size = 128
with requests.get(url, stream=True) as r:
    print(r)
    print(r.headers.keys)
    print(r.headers.get("Content-Length"))
    print(type(r.headers.get("Content-Length")))
    print(type(total_content_length) == str)
    total_content_length = r.headers.get("Content-Length")
    if total_content_length is None:
            with open(save_path, "wb") as fd:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    fd.write(chunk)
    elif type(total_content_length) == str:
         with open(save_path, "wb") as fd:
            with tqdm(
                    total=total_content_length ,
                    unit="B",
                    unit_scale=True,
                    unit_divisor=1024,
                    desc="Downloading Model",
                    initial=0,
                    ascii=True,
                ) as pbar:
                    for chunk in r.iter_content(chunk_size=chunk_size):
                        fd.write(chunk)
                        pbar.update(len(chunk))

In [ ]:
url = 'https://zenodo.org/api/files/70633e17-73c6-4f5e-adb1-062725e6a7c1/sat4class_rgb_512_v1_fullmodel.h5'
with requests.get(url, stream=True) as r:
    print(r)
    print(r.headers)
    print(r.headers.get("Content-Length"))

In [ ]:
downloaded_models_path = r'C:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\src\coastseg\downloaded_models'
dataset_id = 's2-landsat78-4class_6950472'

In [ ]:
zenodo_id = dataset_id.split("_")[-1]
root_url = "https://zenodo.org/api/records/" + zenodo_id
# read raw json and get list of available files in zenodo release
response = requests.get(root_url)
json_content = json.loads(response.text)
files = json_content["files"]

In [ ]:
weights_direc = os.path.abspath(
            os.path.join(downloaded_models_path, dataset_id)
        )
weights_direc

In [ ]:
all_models = [f for f in files if f["key"].endswith(".h5")]
all_models

In [ ]:
all_models[0]["links"]["self"].split('/')[-1]

In [ ]:
all_models[0]["links"]["self"].split(os.sep)[-1]

In [ ]:
best_model_txt = [f for f in files if f["key"] == "BEST_MODEL.txt"][0]
best_model_txt

In [ ]:
best_model_txt_path = weights_direc + os.sep + "BEST_MODEL.txt"
best_model_txt_path

In [ ]:
def download_url(url, save_path, chunk_size=128):
    print(f"url: {url}")
    print(f"save_path: {save_path}")
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [ ]:
def download_url(url: str, save_path: str, chunk_size: int = 128):
    """Downloads the model from the given url to the save_path location.
    Args:
        url (str): url to model to download
        save_path (str): directory to save model
        chunk_size (int, optional):  Defaults to 128.
    """
    # make an HTTP request within a context manager
    with requests.get(url, stream=True) as r:
        # check header to get content length, in bytes
        total_length = int(r.headers.get("Content-Length"))
        with open(save_path, "wb") as fd:
            with tqdm(
                total=total_length,
                unit="B",
                unit_scale=True,
                unit_divisor=1024,
                desc="Downloading Model",
                initial=0,
                ascii=True,
            ) as pbar:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    fd.write(chunk)
                    pbar.update(len(chunk))


In [ ]:
download_url(
                    best_model_txt["links"]["self"],
                    best_model_txt_path,
                )

In [ ]:
with open(best_model_txt_path) as f:
                data = f.read()
data

In [ ]:
best_model_h5 = [f for f in files if f["key"] == data][0]
best_model_h5

In [ ]:
outfile = weights_direc + os.sep + data
outfile

In [ ]:
best_model_json = [
                f for f in files if f["key"] == data.replace("_fullmodel.h5", ".json")
            ]
best_model_json